<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/TF_IDF_Gentzkow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar la base

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
base=pd.read_csv('/content/drive/MyDrive/base_formateada.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])

In [5]:
# Se incorporan los nombres de los oradores como stopwords
from collections import Counter
nombres=[]
for i in base.namey.unique().tolist(): nombres.extend(str(i).split(' '))
nombres = [x.strip(' ') for x in nombres]
nombres = [x.strip(' †') for x in nombres]
nombres = [x.strip(',') for x in nombres]
nombres=list(Counter(nombres))
nombres.remove('')

In [6]:
stopwords.extend(nombres)

In [7]:
partidos=pd.read_csv('/content/drive/MyDrive/partidos.csv', encoding='latin1', sep=';')

In [8]:
stopwords.extend(partidos['nombre'].unique())
stopwords.extend(partidos['1'].unique())
stopwords.extend(partidos['2'].unique())
stopwords.extend(partidos['3'].unique())

In [9]:
len(stopwords)

2483

# TF-IDF mencionadas al menos 100 veces en total

In [67]:
# Recorto a las palabras mencionadas al menos 100 veces en total. Esto ya deja un vocabulario chico de apenas 13mil bigramas. 

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range=[2,2], stop_words=stopwords , min_df=100)
bow = vect.fit_transform(base['tokens'])
total_features = len(vect.vocabulary_)

In [68]:
total_features

13187

In [69]:
matrix = vect.transform(base['tokens'])

In [70]:
matrix.shape

(334421, 13187)

# Mencionadas al menos 10 veces en el día

In [71]:
# Gentzkow et.al. usan mencionadas al menos 10 veces en una "sesión". Acá se usa mencionadas al menos 10 veces en un "día". 
# Esto selecciona apenas mil bigramas
new_voc=[]
vect = TfidfVectorizer(ngram_range=[2,2], stop_words=stopwords , min_df=10)
for x in base.fecha.unique(): 
  base_dia=base[pd.DatetimeIndex(base.fecha)==x]
  try: 
    bow = vect.fit_transform(base_dia['tokens'])
    new_voc+=vect.vocabulary_
  except: pass
  

In [72]:
import numpy as np
len(np.unique(new_voc))

1064

# Combinación

In [73]:
vect = TfidfVectorizer(ngram_range=[2,2], stop_words=stopwords , min_df=100)
bow = vect.fit_transform(base['tokens'])
total_features = len(vect.vocabulary_)

inter=set(vect.vocabulary_).intersection(new_voc)

In [74]:
vect = TfidfVectorizer(ngram_range=[2,2], stop_words=stopwords ,vocabulary=inter)
bow = vect.fit_transform(base['tokens'])
total_features = len(vect.vocabulary_)

In [75]:
# La intersección genera un vocabulario de apenas 814 bigramas. 
total_features

814

In [83]:
matrix = vect.transform(base['tokens'])

# LDA con 10 clusters

In [76]:
from sklearn.decomposition import LatentDirichletAllocation

In [84]:
# Acá hay un problema porque se sigue usando 10 clusters cuando en realidad habría que chequear el número óptimo de clusters una vez definida la cantidad de bigramas a utilizar. 
# Sin embargo, Text as data dice que el número de clústers es generalmente arbitrario, y recomienda arrancar probando con 10. 
lda = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)     

In [85]:
lda.fit(matrix)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, random_state=0)

In [86]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx + 1))
        print(" , ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [87]:
display_topics(lda, feature_names=vect.get_feature_names_out(), no_top_words=20)

Topic 1:
violencia género , proposición popular , disposición adicion , ministra adelant , sáenz santamaría , tramitación proyecto , disposición final , aprobada disposición , procedimiento urgencia , rechazado número , ministra defensa , proyecto procedimiento , lectura única , candidato presidencia , disposición transitoria , parlamento europeo , gonzález txabarri , minoría catalana , hecho referencia , aprobado sección
Topic 2:
vasco pnv , convergència unió , progreso democracia , unión progreso , catalán convergència , ruego guarden , medida urgent , agencia tributaria , pequeña mediana , comisión nacion , mediana empresa , amnistía fiscal , tiempo concluido , fraud fiscal , impuesto renta , ministerio fomento , lucha fraud , persona física , valor añadido , mismo tiempo
Topic 3:
unión europea , per catalunya , catalunya verd , izquierda per , social igualdad , consecuencia interpelación , moción consecuencia , ministra fomento , medio comunicación , propio término , ir acabando , 